In [ ]:
import os

os.environ["SPARK_HOME"] = "C:/Program Files/Spark"
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "lab"
os.environ["PYSPARK_PYTHON"] = (
    "C:/Users//AppData/Local/Programs/Python/Python310/python.exe"
)

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc

# Create a SparkSession
spark = SparkSession.builder.appName("DataFrame-Demo").getOrCreate()

Compare Using RDD's

In [11]:
rdd = spark.sparkContext.textFile("./data/data.txt")
result_rdd = (
    rdd.flatMap(lambda line: line.split(" "))
    .map(lambda word: (word, 1))
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda x: x[1], ascending=False)
)

In [12]:
result_rdd.take(10)

[('the', 12),
 ('of', 7),
 ('a', 7),
 ('in', 5),
 ('distributed', 5),
 ('Spark', 4),
 ('is', 3),
 ('as', 3),
 ('API', 3),
 ('on', 3)]

With Using DataFrames

In [13]:
df = spark.read.text("./data/data.txt")
print(df.show())

result_df = (
    df.selectExpr("explode(split(value, ' ')) as word")
    .groupBy("word")
    .count()
    .orderBy(desc("count"))
)

+--------------------+
|               value|
+--------------------+
|Apache Spark has ...|
|                    |
|Spark and its RDD...|
+--------------------+

None


In [14]:
result_df.take(10)

[Row(word='the', count=12),
 Row(word='of', count=7),
 Row(word='a', count=7),
 Row(word='in', count=5),
 Row(word='distributed', count=5),
 Row(word='Spark', count=4),
 Row(word='API', count=3),
 Row(word='RDD', count=3),
 Row(word='is', count=3),
 Row(word='on', count=3)]

In [15]:
spark.stop()